# Installing Dependencies.

In [ ]:
!pip install langchain
!pip install openai
!pip install pinecone-client
!pip install jq
!pip install tiktoken

# Importing Dependencies.

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# API keys and Env.

In [3]:
openai_api_key = ''

PINECONE_API_KEY = ''

PINECONE_API_ENV = ''

# Loading the data.

In [5]:
import json
from pathlib import Path
from pprint import pprint


file_path='/kaggle/input/json-dataset-of-people/Customer data.json'
data = json.loads(Path(file_path).read_text())

In [6]:
pprint(data[:5])

[{'Ethnicity': 'White',
  'Location': 'Guararapes',
  'Occupation': 'Nuclear Power Engineer',
  'email': 'pmacmaster0@bandcamp.com',
  'first_name': 'Prince',
  'gender': 'Male',
  'id': 1,
  'ip_address': '208.24.50.172',
  'last_name': 'MacMaster'},
 {'Ethnicity': 'Tongan',
  'Location': 'Fram',
  'Occupation': 'Health Coach II',
  'email': 'csantori1@wisc.edu',
  'first_name': 'Caryl',
  'gender': 'Female',
  'id': 2,
  'ip_address': '151.160.135.93',
  'last_name': 'Santori'},
 {'Ethnicity': 'Delaware',
  'Location': 'Bandera',
  'Occupation': 'VP Product Management',
  'email': 'ccadle2@flavors.me',
  'first_name': 'Cort',
  'gender': 'Male',
  'id': 3,
  'ip_address': '196.150.14.252',
  'last_name': 'Cadle'},
 {'Ethnicity': 'Vietnamese',
  'Location': 'Ciechocin',
  'Occupation': 'Assistant Media Planner',
  'email': 'pchampagne3@ft.com',
  'first_name': 'Peg',
  'gender': 'Female',
  'id': 4,
  'ip_address': '74.183.119.175',
  'last_name': 'Champagne'},
 {'Ethnicity': 'Yakama'

# Obtaining the names for our 'ids' and vector generation part.

In [ ]:

names_list = []
for json_obj in data:
  name = json_obj["first_name"] + " " + json_obj["last_name"]
  names_list.append(name)
names_list

# changing the json to a dictionary.

In [ ]:
import json

# Read the JSON file
with open('/kaggle/input/json-dataset-of-people/Customer data.json', 'r') as file:
    json_data = file.read()

# Parse the JSON data into a dictionary
data_dict = json.loads(json_data)

# Now you can work with the data_dict as a regular Python dictionary
print(data_dict)


# Removing the names from our metadata list.

In [23]:

for obj in data_dict:
    del obj['first_name']
    del obj['last_name']


# Embedding the documents Using Pinecone.

## For this type of upsertion,we embed the names of the customers,so that our insertion format would look like (ID(names),VECTORS(embedded names),METADATA(the restof the data in dictionary format)

In [9]:
# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [10]:
n_embeddings = embeddings.embed_documents(names_list)

In [12]:
len(n_embeddings[0])

1536

# Combining them both gives the (id,vector,metadata) tuple that can now be upserted to pinecone.

In [27]:
combined_list = [(names_list[i], n_embeddings[i], data_dict[i]) for i in range(len(names_list))]

In [ ]:
combined_list[0]

# The data needs to in 100 batches to be upserted.
## Splitting the data and storing them in one list.

In [29]:
main_list=[]
first_100 = combined_list[:100]
main_list.append(first_100)
second_100 = combined_list[100:200]
main_list.append(second_100)
third_100 = combined_list[200:300]
main_list.append(third_100)
fourth_100 = combined_list[300:400]
main_list.append(fourth_100)
fifth_100 = combined_list[400:500]
main_list.append(fifth_100)
sixth_100 = combined_list[500:600]
main_list.append(sixth_100)
seventh_100 = combined_list[600:700]
main_list.append(seventh_100)
eighth_100 = combined_list[700:800]
main_list.append(eighth_100)
ninth_100 = combined_list[800:900]
main_list.append(ninth_100)
tenth_100 = combined_list[900:1000]
main_list.append(tenth_100)


In [30]:
print(len(first_100))
print(len(tenth_100))

100
100


# The length of the list should be 10

In [31]:
len(main_list)

10

In [32]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "metadata-insert" # put in the name of your pinecone index here

# Instantiating the index.

In [34]:
index = pinecone.Index("metadata-insert")

# Can only Upsert data in batches of 100.

In [35]:
for i in main_list:
    index.upsert(
        i
    )

# Describing the index.


In [37]:
pinecone.describe_index("metadata-insert")

IndexDescription(name='metadata-insert', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')